In [ ]:
from urllib.request import urlopen
from xml.etree.ElementTree import parse

u = open('./planet_python.xml')

In [ ]:
doc = parse(u)

In [ ]:
for item in doc.iterfind('channel/item'):
    title = item.findtext('title')
    date = item.findtext("pubDate")
    link = item.findtext('link')

    print("*" * 10)
    print(title)
    print(date)
    print(link)
    print("*" * 10)

In [ ]:
for element in doc.iterfind('channel/item'):
    print(element.findtext("title"))

In [48]:
from xml.etree.ElementTree import iterparse

data = iterparse('./planet_python.xml', ('start-ns', 'end-ns'))
next(data), next(data)

(('start-ns', ('dc', 'http://purl.org/dc/elements/1.1/')), ('end-ns', None))

In [ ]:
# import urlparser from standard library
from xml.etree.ElementTree import iterparse

def parse_and_remove(filename, path):
    path_parts = path.split('/')

    doc = iterparse(filename, ('start', 'end'))

    # skip the root element
    next(doc)

    tag_stack = []
    elem_stack = []

    for event, elem in doc:
        if event == 'start':
            tag_stack.append(elem.tag)
            elem_stack.append(elem)
        elif event == 'end':
            if tag_stack == path_parts:
                yield elem
                elem_stack[-2].remove(elem)
            try:
                tag_stack.pop()
                elem_stack.pop()
            except IndexError:
                pass
        

: 

In [4]:
from xml.etree.ElementTree import Element, tostring

def dict_to_xml(tag, d):
    elem = Element(tag)
    for key, value in d.items():
        child = Element(key)
        child.text = value
        elem.append(child)
    return elem

s = {"name": "Google", "shares": '100', "price": '490.1'}
e = dict_to_xml('stock', s)
e.set('_id', '1234')
tostring(e)

b'<stock _id="1234"><name>Google</name><shares>100</shares><price>490.1</price></stock>'

In [10]:
def dict_to_xml_v2(tag, d):
    parts = ['<{}>'.format(tag)]

    for key, val in d.items():
        parts.append(f"<{key}>{val}</{key}>")
    parts.append(f"</{tag}>")
    return ''.join(parts)
s = { 'name' : '<spam>' }
tostring(dict_to_xml('stock', s))


b'<stock><name>&lt;spam&gt;</name></stock>'

In [12]:
from xml.sax.saxutils import escape, unescape

escape('<spam>')

'&lt;spam&gt;'

In [55]:
from xml.etree.ElementTree import parse, Element

doc = parse('./test.xml')

root = doc.getroot()
for pre_element in root.findall('pre'):
    pt_element = pre_element.find('pt')
    childrens = list(pre_element)
    # element.get_children is not more availiable
    if pt_element is not None:
        new_element = Element('new_element')
        new_element.text = 'This is a new element'
        pre_element.insert(childrens.index(pt_element) + 1, new_element)

doc.write('./test2.xml', xml_declaration=True)

In [29]:
print(tostring(root))

b'<stop>\n    <id>14791</id>\n    <nm>Clark &amp; Balmoral</nm>\n    <sri>\n        <rt>22</rt>\n        <d>North Bound</d>\n        <dd>North Bound</dd>\n    </sri>\n    <cr>22</cr>\n    <pre>\n        <pt>5 MIN</pt>\n        <fd>Howard</fd>\n        <v>1378</v>\n        <rn>22</rn>\n    </pre>\n    <pre>\n        <pt>15 MIN</pt>\n        <fd>Howard</fd>\n        <v>1867</v>\n        <rn>22</rn>\n    </pre>\n</stop>'


In [ ]:
root.find('pre').find('pt')

<Element 'pt' at 0x7f8680c71ee0>

In [2]:
from xml.etree.ElementTree import parse, Element

doc = parse('./test_xml2.xml')
doc.findtext('author')


'David Beazley'

In [15]:
doc.findtext('content/html')
# willnot work because of the namespace

element = doc.findtext('content/{http://www.w3.org/1999/xhtml}html/{http://www.w3.org/1999/xhtml}head/{http://www.w3.org/1999/xhtml}title')
element

'Hello World'

In [17]:
class XMLNamespaces:
    def __init__(self, **kwargs):
        self.namespaces = {}
        for name, uri in kwargs.items():
            self.register(name, uri)
        
    def register(self, name, uri):
        self.namespaces[name] = "{%s}" % uri
    
    def __call__(self, path):
        return path.format_map(self.namespaces)

ns = XMLNamespaces(html='http://www.w3.org/1999/xhtml')
doc.find(ns('content/{html}html'))

<Element '{http://www.w3.org/1999/xhtml}html' at 0x7f04045868e0>

In [19]:
string = 'Hello {world}'
string.format_map({'world': 'Earth'})

'Hello Earth'

In [23]:
import sqlite3


stocks = [
    ('GOOG', 100, 490.1),
    ('AAPL', 50, 545.75),
    ('FB', 150, 7.45),
    ('HPQ', 75, 33.2)
]


db = sqlite3.connect('database.db')

c = db.cursor()
c.execute(
    """
        create table portfolio (symbol text, shares integer, price real)
    """
)
db.commit()

c.executemany(
    """
        insert into portfolio values (?, ?, ?)
    """, stocks
)
db.commit()

for symbol, shares, price in db.execute('select * from portfolio'):
    print(symbol, shares, price)

# GOOG 100 490.1
# AAPL 50 545.75
# FB 150 7.45
# HPQ 75 33.2


OperationalError: table portfolio already exists

In [30]:
from binascii import b2a_hex, a2b_hex
import base64


s = b'hello'
print(b2a_hex(s), base64.b16encode(s))
print(base64.b64decode(base64.b64encode(s).decode('ascii')))

b'68656c6c6f' b'68656C6C6F'
b'hello'


In [4]:
import emoji

# Function to convert a string to emojis
def string_to_emoji(input_str):
    emoji_str = ''
    for char in input_str:
        emoji_char = emoji.emojize(char)
        if emoji_char:
            emoji_str += emoji_char + ' '
        else:
            emoji_str += char
    return emoji_str

# Function to convert emojis back to a string
def emoji_to_string(emoji_str):
    parts = emoji_str.split()
    text = ''
    for part in parts:
            text += part
    return text

# Example usage
input_string = "Hello, World! 🌍"
emoji_string = string_to_emoji(input_string)
print(f"String to Emoji: {emoji_string}")
decoded_string = emoji_to_string(emoji_string)
print(f"Emoji to String: {decoded_string}")

String to Emoji: H e l l o ,   W o r l d !   🌍 
Emoji to String: Hello,World!🌍


In [29]:
# Function to convert a string to emojis based on ASCII or UTF values
start_offset = 0x1F600  # Start of Emojis in Unicode table
end_offset = 0x1F64F  # End of Emojis in Unicode table

# Function to convert a string to emojis based on Unicode code points
def string_to_emoji(input_str):
    emoji_str = ''
    
    for char in input_str:
        # if character is already an emoji, then convert it to another emoji
        if start_offset <= ord(char) <= end_offset:
            # need to handle situation to identity if character is already an emoji
            # if so, then convert it to another emoji and to convert it back to a string


            ord_value = ord(char) - start_offset
            emoji_str += chr(ord_value + start_offset)
        else:
            emoji_str += chr(ord(char) + start_offset)

    return emoji_str

# Function to convert emojis back to a string
def emoji_to_string(emoji_str):
    text = ''
    
    for char in emoji_str:
        # if character is an emoji, then convert it to another character
        if start_offset <= ord(char) <= end_offset:
            ord_value = ord(char) - start_offset
            text += chr(ord_value + start_offset)
        else:
            text += chr(ord(char) - start_offset)

    return text

# Offset for converting emojis
offset = 128512

# Function to convert a string to emojis based on Unicode code points
def string_to_emoji(input_str):
    emoji_str = ''

    for char in input_str:
        if ord(char) >= offset:
            # Character is an emoji, convert it to another emoji
            encoded_char = ord(char) ^ offset
            emoji_str += chr(encoded_char)
        else:
            encoded_char = ord(char) ^ offset
            emoji_str += chr(encoded_char)

    return emoji_str

# Function to convert emojis back to a string
def emoji_to_string(emoji_str):
    text = ''

    for char in emoji_str:
        decoded_char = ord(char) ^ offset
        text += chr(decoded_char)

    return text

# Example usage
input_string = "Hello, World! 😂🌍"
emoji_string = string_to_emoji(input_string)
print(f"String to Emoji: {emoji_string}")
decoded_string = emoji_to_string(emoji_string)
print(f"Emoji to String: {decoded_string}")

String to Emoji: 🙈🙥🙬🙬🙯😬😠🙗🙯🙲🙬🙤😡😠😀ԍ
Emoji to String: Hello, World!  🌍


'😐'